# 第4章: 言語解析
問題30から問題35までは、以下の文章`text`（太宰治の『走れメロス』の冒頭部分）に対して、言語解析を実施せよ。問題36から問題39までは、国家を説明した文書群（日本語版ウィキペディア記事から抽出したテキスト群）をコーパスとして、言語解析を実施せよ。
```
text = """
メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
"""

In [5]:
!brew install mecab
!brew install mecab-ipadic

==> Auto-updating Homebrew...
Adjust how often this is run with `$HOMEBREW_AUTO_UPDATE_SECS` or disable with
`$HOMEBREW_NO_AUTO_UPDATE=1`. Hide these hints with `$HOMEBREW_NO_ENV_HINTS=1` (see `man brew`).
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
attempt-cli: CLI for retrying fallible commands
bedtk: Simple toolset for BED files
claude-cmd: Claude Code Commands Manager
claude-hooks: Hook system for Claude Code
dash-mpd-cli: Download media content from a DASH-MPEG or DASH-WebM MPD manifest
fjira: Fuzzy-find cli jira interface
komac: Community Manifest Creator for Windows Package Manager (WinGet)
mcp-server-kubernetes: MCP Server for kubernetes management commands
oatpp: Light and powerful C++ web framework
opencode: AI coding agent, built for the terminal
terratag: CLI to automate tagging for AWS, Azure & GCP resources in Terraform
tweakcc: Customize your Claude Code themes, thinking verbs, and more
wal-g: Archival restoration tool fo

In [6]:
!pip install MeCab-Python3


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


# 30. 動詞
文章`text`に含まれる動詞をすべて表示せよ。

In [2]:
text = """メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。"""

In [13]:
import MeCab

mecab = MeCab.Tagger()
parsed = mecab.parse(text)

mecab_list = []
for line in parsed.split("\n"):
    a_word = line.strip().split("\t")
    if len(a_word) == 1 and a_word[0] == "EOS":
        break
    elif len(a_word) == 2:
        right_side = a_word[1].split(",")
        if len(right_side) > 6:
            mecab_list.append({
                "surface": a_word[0],
                "base": right_side[6],
                "pos": right_side[0],
                "pos[1]": right_side[1]
            })

mecab_list

[{'surface': 'メロス', 'base': '*', 'pos': '名詞', 'pos[1]': '一般'},
 {'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos[1]': '係助詞'},
 {'surface': '激怒', 'base': '激怒', 'pos': '名詞', 'pos[1]': 'サ変接続'},
 {'surface': 'し', 'base': 'する', 'pos': '動詞', 'pos[1]': '自立'},
 {'surface': 'た', 'base': 'た', 'pos': '助動詞', 'pos[1]': '*'},
 {'surface': '。', 'base': '。', 'pos': '記号', 'pos[1]': '句点'},
 {'surface': '必ず', 'base': '必ず', 'pos': '副詞', 'pos[1]': '助詞類接続'},
 {'surface': '、', 'base': '、', 'pos': '記号', 'pos[1]': '読点'},
 {'surface': 'かの', 'base': 'かの', 'pos': '連体詞', 'pos[1]': '*'},
 {'surface': '邪智', 'base': '邪智', 'pos': '名詞', 'pos[1]': '一般'},
 {'surface': '暴虐', 'base': '暴虐', 'pos': '名詞', 'pos[1]': '一般'},
 {'surface': 'の', 'base': 'の', 'pos': '助詞', 'pos[1]': '連体化'},
 {'surface': '王', 'base': '王', 'pos': '名詞', 'pos[1]': '一般'},
 {'surface': 'を', 'base': 'を', 'pos': '助詞', 'pos[1]': '格助詞'},
 {'surface': '除か', 'base': '除く', 'pos': '動詞', 'pos[1]': '自立'},
 {'surface': 'なけれ', 'base': 'ない', 'pos': '助動詞', 'pos[1]': '*'}

In [16]:
verb_list = []

for item in mecab_list:
    if item["pos"] == "動詞":
        verb_list.append(item["surface"])
verb_list

['し', '除か', 'なら', 'し', 'わから', '吹き', '遊ん', '暮し', '来']

# 31. 動詞の原型
文章`text`に含まれる動詞と、その原型をすべて表示せよ。

In [19]:
verb_base_list = []

for item in mecab_list:
    if item["pos"] == "動詞":
        verb_base_list.append({"surface" : item["surface"], "base" : item["base"]})
verb_base_list

[{'surface': 'し', 'base': 'する'},
 {'surface': '除か', 'base': '除く'},
 {'surface': 'なら', 'base': 'なる'},
 {'surface': 'し', 'base': 'する'},
 {'surface': 'わから', 'base': 'わかる'},
 {'surface': '吹き', 'base': '吹く'},
 {'surface': '遊ん', 'base': '遊ぶ'},
 {'surface': '暮し', 'base': '暮す'},
 {'surface': '来', 'base': '来る'}]

# 32. 「AのB」
文章`text`において、2つの名詞が「の」で連結されている名詞句をすべて抽出せよ。

In [21]:
result = set()

for sentence in mecab_map:
    for i in range(len(sentence)-2):
        adjacent_morphs = sentence[i:i+3]
        if (adjacent_morphs[0]["pos"] == adjacent_morphs[2]["pos"] == "名詞") \
            and adjacent_morphs[1]["surface"] == "の": 
            connected_noun = "".join(map(lambda x:x["surface"], adjacent_morphs))
            result.add(connected_noun)

result

{'暴虐の王', '村の牧人'}

# 33. 係り受け解析
文章`text`に係り受け解析を適用し、係り元と係り先のトークン（形態素や文節などの単位）をタブ区切り形式ですべて抽出せよ。

In [24]:
!brew install cabocha
!pip install cabocha-python

To reinstall 0.69, run:
  brew reinstall cabocha

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [31]:
import CaboCha

c = CaboCha.Parser()

dep_list = []  

for sentence in text.split("。"):
    sentence = sentence.strip()
    if not sentence:
        continue
    tree = c.parse(sentence)

    for i in range(tree.size()):
        token = tree.token(i)
        chunk = token.chunk
        if chunk:
            if chunk.link >= 0:  
                src = "".join([tree.token(k).surface for k in range(chunk.token_pos, chunk.token_pos + chunk.token_size)])
                dst_chunk = tree.chunk(chunk.link)
                dst = "".join([tree.token(k).surface for k in range(dst_chunk.token_pos, dst_chunk.token_pos + dst_chunk.token_size)])
                dep_list.append([src, dst])

for pair in dep_list:
    print(pair[0], "\t", pair[1])

メロスは 	 激怒した
必ず、 	 決意した
かの 	 邪智暴虐の
邪智暴虐の 	 王を
王を 	 除かなければならぬと
除かなければならぬと 	 決意した
メロスには 	 わからぬ
政治が 	 わからぬ
メロスは、 	 牧人である
村の 	 牧人である
笛を 	 吹き、
吹き、 	 暮して来た
羊と 	 遊んで
遊んで 	 暮して来た
けれども 	 敏感であった
邪悪に対しては、 	 敏感であった
人一倍に 	 敏感であった


# 34. 主述の関係
文章`text`において、「メロス」が主語であるときの述語を抽出せよ。

In [35]:
import MeCab

m = MeCab.Tagger()

predicates = []

for src, dst in dep_list:
    if "メロス" in src:
        if src.endswith("は") or src.endswith("が"):
            node = m.parseToNode(dst)
            has_verb = False
            while node:
                features = node.feature.split(",")
                if features[0] == "動詞":
                    has_verb = True
                    break
                node = node.next
            if has_verb:
                predicates.append(dst)

print(predicates)

['激怒した', 'わからぬ']


# 35. 係り受け木
「メロスは激怒した。」の係り受け木を可視化せよ。

# 36. 単語の出現頻度
問題36から39までは、Wikipediaの記事を以下のフォーマットで書き出したファイル[jawiki-country.json.gz](https://nlp100.github.io/2025/_downloads/ca47e7baf341469cd7b585f97496c020/jawiki-country.json.gz)をコーパスと見なし、統計的な分析を行う。

 1行に1記事の情報がJSON形式で格納される

各行には記事名が”title”キーに、記事本文が”text”キーの辞書オブジェクトに格納され、そのオブジェクトがJSON形式で書き出される

ファイル全体はgzipで圧縮される

まず、第3章の処理内容を参考に、Wikipedia記事からマークアップを除去し、各記事のテキストを抽出せよ。そして、コーパスにおける単語（形態素）の出現頻度を求め、出現頻度の高い20語とその出現頻度を表示せよ。